In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#printing stopwords 
## Stopwords have no significance 
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
### Loading Data
twitter_data_train = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv')
twitter_data_test = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv')

In [6]:
twitter_data_train.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [7]:
# renaming the columns
column_names = ['id','Not_sure_column','target','Tweet']

twitter_data_train = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv',names = column_names)
twitter_data_test = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv',names = column_names)

In [8]:
twitter_data_train.head()

,id,Not_sure_column,target,Tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [9]:
twitter_data_train=twitter_data_train.replace({'target':{'Irrelevant':2,'Positive':1,'Negative':0,'Neutral':2}})

/tmp/ipykernel_33/821463325.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  twitter_data_train=twitter_data_train.replace({'target':{'Irrelevant':2,'Positive':1,'Negative':0,'Neutral':2}})


In [23]:
twitter_data_test=twitter_data_test.replace({'target':{'Irrelevant':2,'Positive':1,'Negative':0,'Neutral':2}})

/tmp/ipykernel_33/3563915277.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  twitter_data_test=twitter_data_test.replace({'target':{'Irrelevant':2,'Positive':1,'Negative':0,'Neutral':2}})


In [10]:
### Counting missing tweets
twitter_data_train.isnull().sum()

id                   0
Not_sure_column      0
target               0
Tweet              686
dtype: int64

In [24]:
### Removing null rows
twitter_data_train=twitter_data_train.dropna()
twitter_data_test=twitter_data_test.dropna()

In [25]:
X_train = twitter_data_train['Tweet']
Y_train = twitter_data_train['target']

X_test = twitter_data_test['Tweet']
Y_test = twitter_data_test['target']

In [26]:
## Converting Positive as 3 Neutral as 2 and 1 as negative
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
Y_train=le.fit_transform(Y_train)
Y_test = le.fit_transform(Y_test)

In [14]:
## checking the distribution of target column
twitter_data_train['target'].value_counts()

target
2    30983
0    22358
1    20655
Name: count, dtype: int64

In [27]:
twitter_data_train.head()

,id,Not_sure_column,target,Tweet,stemmed_content
0,2401,Borderlands,1,im getting on borderlands and i will murder yo...,im get borderland murder
1,2401,Borderlands,1,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,1,im getting on borderlands and i will kill you ...,im get borderland kill
3,2401,Borderlands,1,im coming on borderlands and i will murder you...,im come borderland murder
4,2401,Borderlands,1,im getting on borderlands 2 and i will murder ...,im get borderland murder


#### Stemming

In [16]:
port_stem = PorterStemmer()

In [17]:
def stemming(content):
    
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content

In [18]:
twitter_data_train['stemmed_content'] = twitter_data_train['Tweet'].apply(stemming)

In [28]:
twitter_data_test['stemmed_content'] = twitter_data_test['Tweet'].apply(stemming)

In [19]:
twitter_data_train.head()

,id,Not_sure_column,target,Tweet,stemmed_content
0,2401,Borderlands,1,im getting on borderlands and i will murder yo...,im get borderland murder
1,2401,Borderlands,1,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,1,im getting on borderlands and i will kill you ...,im get borderland kill
3,2401,Borderlands,1,im coming on borderlands and i will murder you...,im come borderland murder
4,2401,Borderlands,1,im getting on borderlands 2 and i will murder ...,im get borderland murder


In [38]:
X_train = list(twitter_data_train['stemmed_content'].values)
Y_train = list(twitter_data_train['target'].values)

X_test = list(twitter_data_test['stemmed_content'].values)
Y_test = list(twitter_data_test['target'].values)

In [39]:
# converting textual data to numerical
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [42]:
print(X_train)

  (0, 12545)	0.672460959876397
  (0, 2122)	0.4325675736546788
  (0, 7341)	0.32489583024771473
  (0, 9088)	0.5050982589083702
  (1, 10336)	0.4966318265888706
  (1, 2120)	0.7452188611559063
  (1, 3458)	0.4449782891293641
  (2, 10336)	0.5345301888165152
  (2, 2122)	0.49394422039680846
  (2, 7341)	0.3709950244906562
  (2, 9088)	0.5767662231645913
  (3, 3458)	0.4054038888193035
  (3, 12545)	0.6499836346024973
  (3, 2122)	0.41810879814782276
  (3, 9088)	0.48821511098131337
  (4, 12545)	0.672460959876397
  (4, 2122)	0.4325675736546788
  (4, 7341)	0.32489583024771473
  (4, 9088)	0.5050982589083702
  (5, 12545)	0.672460959876397
  (5, 2122)	0.4325675736546788
  (5, 7341)	0.32489583024771473
  (5, 9088)	0.5050982589083702
  (6, 9798)	0.25052583390447347
  (6, 21164)	0.28974429718140715
  :	:
  (73993, 15566)	0.3088104796360347
  (73993, 21910)	0.20383219841601852
  (73994, 13997)	0.42400408152426794
  (73994, 13240)	0.2020200182081118
  (73994, 5128)	0.2876604671630894
  (73994, 21313)	0.2717926

In [44]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [46]:
X_train_predict = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_predict)

In [47]:
training_data_accuracy

0.8586004648899941

In [48]:
X_test_predict = model.predict(X_test)

In [49]:
testing_data_Accuracy = accuracy_score(Y_test,X_test_predict)

In [50]:
testing_data_Accuracy

0.895